## 1、生成数据集

In [3]:
import numpy as np
import torch
from torch.utils import data
# from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2



def synthetic_data(w, b, num_examples):  #@save
    """生成 y = Xw + b + 噪声。"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
#     print(y.shape)
    y += torch.normal(0, 0.01, y.shape) #torch.normal(means, std, out=None)
#     print(X)
#     print(y)
    return X, y.reshape((-1, 1))


# features, labels = d2l.synthetic_data(true_w, true_b, 1000)
features, labels = synthetic_data(true_w, true_b, 1000)

## 2、读取数据集

In [4]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器。"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [5]:
next(iter(data_iter))

[tensor([[ 1.4695,  1.0947],
         [ 1.4480,  1.0072],
         [ 0.6078,  1.1212],
         [ 1.7217,  0.2664],
         [ 0.6093,  1.6815],
         [-1.2784, -0.3523],
         [-0.9550, -1.2661],
         [ 0.7224,  2.1331],
         [ 0.6544,  0.9155],
         [ 0.4345,  0.5981]]),
 tensor([[ 3.4030],
         [ 3.6709],
         [ 1.6090],
         [ 6.7467],
         [-0.2840],
         [ 2.8313],
         [ 6.5723],
         [-1.6186],
         [ 2.3873],
         [ 3.0139]])]

## 3、定义模型

在 PyTorch 中，全连接层在 Linear 类中定义；

值得注意的是，我们将两个参数传递到 nn.Linear 中。第一个指定输入特征形状，即 2，第二个指定输出特征形状，输出特征形状为单个标量，因此为 1。

In [6]:
# `nn` 是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

## 4、初始化模型参数
正如我们在构造 nn.Linear 时指定输入和输出尺寸一样。现在我们直接访问参数以设定初始值。我们通过 net[0] 选择网络中的第一个图层，然后使用 weight.data 和 bias.data 方法访问参数。然后使用替换方法 normal_ 和 fill_ 来重写参数值。

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 5、定义损失函数

计算均方误差使用的是MSELoss类，也称为平方  L2  范数。默认情况下，它返回所有样本损失的平均值。

In [8]:
loss = nn.MSELoss()

## 6、定义优化算法

小批量随机梯度下降算法是一种优化神经网络的标准工具，PyTorch 在 optim 模块中实现了该算法的许多变种。当我们实例化 SGD 实例时，我们要指定优化的参数（可通过 net.parameters() 从我们的模型中获得）以及优化算法所需的超参数字典。小批量随机梯度下降只需要设置 lr值，这里设置为 0.03。

In [9]:
import torch
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 7、训练

在每个迭代周期里，我们将完整遍历一次数据集（train_data），不停地从中获取一个小批量的输入和相应的标签。对于每一个小批量，我们会进行以下步骤:

    通过调用 net(X) 生成预测并计算损失 l（正向传播）。

    通过进行反向传播来计算梯度。

    通过调用优化器来更新模型参数。

为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000355
epoch 2, loss 0.000096
epoch 3, loss 0.000095


In [11]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0002, 0.0002])
b的估计误差： tensor([0.0009])
